# Wisconsin faculty salaries

In this demo, we'll analyze data on average faculty salaries in Wisconsin, as compiled by the [Chronicle of Higher Education](http://data.chronicle.com/category/state/Wisconsin/faculty-salaries/).  The file ```wisconsinfaculty.txt``` contains data on faculty salaries at the 25 highest-paying institutions in Wisconsin in the 2013-2014 academic year, ordered by pay for faculty with the rank of Professor.

In [1]:
import pandas as pd
import numpy as np

To start, we look at the first 10 lines of the file, to understand its structure.

Instead of uploading the data file to Jupyter, we specify a file path on our computer.
This is helpful when working with large files, because Jupyter won't allow us to upload files that are larger than 25 Mb.

In [2]:
# Change the value of fileLoc to match where you saved the file.
fileLoc = "wisconsinfaculty.txt"
with open(fileLoc) as wisc_file:
    line_num = 0
    for line in wisc_file:
        print("Line number:", line_num, line)
        line_num +=1
        if line_num==10:
            brea
            k
        

Line number: 0 Institution	Professors	Associate Professors	Assistant Professors	Instructors	Lecturers	Unranked

Line number: 1 Marquette University

Line number: 2 Wisconsin | 4-year private | Doctoral	$118,953	$83,853	$72,702	$55,863	$53,937	$56,052

Line number: 3 University of Wisconsin at Madison

Line number: 4 Wisconsin | 4-year public | Very high research activity	$113,094	$84,510	$77,157	$51,120	$66,078	$34,407

Line number: 5 University of Wisconsin at Milwaukee

Line number: 6 Wisconsin | 4-year public | High research activity	$97,317	$74,925	$70,173	$49,077	$45,513	--

Line number: 7 Lawrence University

Line number: 8 Wisconsin | 4-year private | Arts and sciences baccalaureate	$90,324	$69,597	$54,396	$50,859	--	--

Line number: 9 Carthage College



We see that after a header line, institution names are on odd lines and institution data is on even lines.

We want to build a data frame with the following headers: Institution, Is Public, Professors, Associate Professors	Assistant Professors, Instructors, Lecturers, and Unranked.

Let's start by making sure we can extract the appropriate information.  We'll test our code on the first few lines of the file, to make sure it works.

In [3]:
%%time
with open(fileLoc) as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        print(inst, is_public, salary_list)
            
        line_num += 2

Marquette University False [118953, 83853, 72702, 55863, 53937, 56052]
University of Wisconsin at Madison True [113094, 84510, 77157, 51120, 66078, 34407]
University of Wisconsin at Milwaukee True [97317, 74925, 70173, 49077, 45513, nan]
Lawrence University False [90324, 69597, 54396, 50859, nan, nan]
Carthage College False [84501, 68958, 57816, 34128, nan, nan]
Wall time: 3.99 ms


Now we have to insert our information in a data frame.  A naive way to do this is to create an empty data frame and append the information for each row as we compute it.

In [4]:
%%time 
with open(fileLoc) as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    # create an empty data frame with the appropriate columns
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    fac_df = pd.DataFrame(columns=column_names)
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        fac_df = fac_df.append(pd.DataFrame([[inst, is_public]+salary_list], columns = column_names))
            
        line_num += 2

print(fac_df)

                            Institution Is Public Professors  \
0                  Marquette University     False     118953   
0    University of Wisconsin at Madison      True     113094   
0  University of Wisconsin at Milwaukee      True      97317   
0                   Lawrence University     False      90324   
0                      Carthage College     False      84501   

  Associate Professors Assistant Professors Instructors Lecturers Unranked  
0                83853                72702       55863     53937    56052  
0                84510                77157       51120     66078    34407  
0                74925                70173       49077     45513      NaN  
0                69597                54396       50859       NaN      NaN  
0                68958                57816       34128       NaN      NaN  
Wall time: 143 ms


We see that appending the information for each row is significantly slower than merely computing the information itself.  The problem is that appending data frames is a memory-intensive operation.

We can make our code faster by storing the information we compute in a list of dictionaries, and converting to a data frame at the end.

In [5]:
%%time
with open(fileLoc) as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    dict_list = []
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    
    line_num=1
    while line_num < 10:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        dict_list.append(dict(zip(column_names, [inst,is_public]+salary_list)))
            
        line_num += 2

fac_df = pd.DataFrame(dict_list)
print(fac_df)

   Assistant Professors  Associate Professors  \
0                 72702                 83853   
1                 77157                 84510   
2                 70173                 74925   
3                 54396                 69597   
4                 57816                 68958   

                            Institution  Instructors  Is Public  Lecturers  \
0                  Marquette University        55863      False    53937.0   
1    University of Wisconsin at Madison        51120       True    66078.0   
2  University of Wisconsin at Milwaukee        49077       True    45513.0   
3                   Lawrence University        50859      False        NaN   
4                      Carthage College        34128      False        NaN   

   Professors  Unranked  
0      118953   56052.0  
1      113094   34407.0  
2       97317       NaN  
3       90324       NaN  
4       84501       NaN  
Wall time: 17 ms


We're ready to run our code on the full file!

In [6]:
with open(fileLoc) as wisc_file:
    
    #skip the header line
    next(wisc_file)
    
    dict_list = []
    column_names = ('Institution', 'Is Public', 'Professors', 'Associate Professors', 'Assistant Professors', 'Instructors', 'Lecturers', 'Unranked')
    
    #We rely on our break command at the end of the file to exit the loop gracefully
    
    while True:
        # We use readline() to read two lines at once
        odd_line = wisc_file.readline()
        even_line = wisc_file.readline()
        
        #break out of the loop if we hit the end of the file
        if not even_line:
            break
        
        inst = odd_line.strip()
        
        #split the even line on the pipe symbol to extract public vs. private
        pipe_list = even_line.split("|")
        is_public =  "public" in pipe_list[1]
        
        #split on \t to extract the salary amount
        salary_list = pipe_list[2].split("\t")[1:]
        
        #strip special characters from salaries
        salary_list = [s.translate({ord(i):None for i in "$,\n-"}) for s in salary_list]
        
        #convert strings to numbers if non-empty
        salary_list = [np.nan if s=="" else eval(s) for s in salary_list]
        
        dict_list.append(dict(zip(column_names, [inst,is_public]+salary_list)))

fac_df = pd.DataFrame(dict_list)
fac_df

,Assistant Professors,Associate Professors,Institution,Instructors,Is Public,Lecturers,Professors,Unranked
0,72702,83853,Marquette University,55863.0,False,53937.0,118953,56052.0
1,77157,84510,University of Wisconsin at Madison,51120.0,True,66078.0,113094,34407.0
2,70173,74925,University of Wisconsin at Milwaukee,49077.0,True,45513.0,97317,NaN
3,54396,69597,Lawrence University,50859.0,False,NaN,90324,NaN
4,57816,68958,Carthage College,34128.0,False,NaN,84501,NaN
5,69642,73674,Milwaukee School of Engineering,54603.0,False,NaN,83034,NaN
6,54828,65718,Carroll University,NaN,False,51273.0,81315,NaN
7,52956,69543,Saint Norbert College,33822.0,False,NaN,81018,47052.0
8,65610,64980,University of Wisconsin at Whitewater,NaN,True,48186.0,78966,NaN
9,57789,62649,University of Wisconsin at La Crosse,NaN,True,42588.0,78561,NaN


Now that we have created the data frame, it's easy to extract summary statistics!

In [7]:
fac_df.describe()

,Assistant Professors,Associate Professors,Instructors,Lecturers,Professors,Unranked
count,25.000000,25.00000,15.000000,15.000000,25.000000,3.000000
mean,58452.480000,65414.16000,48934.200000,46088.400000,79972.920000,45837.000000
std,7884.770733,7795.29043,13159.404303,6928.117481,12950.308699,10873.530935
min,47133.000000,56475.00000,26856.000000,37755.000000,67860.000000,34407.000000
25%,52956.000000,59121.00000,43884.000000,42016.500000,71199.000000,40729.500000
50%,56439.000000,62649.00000,49077.000000,44667.000000,78048.000000,47052.000000
75%,59526.000000,69543.00000,52861.500000,46980.000000,81315.000000,51552.000000
max,77157.000000,84510.00000,83124.000000,66078.000000,118953.000000,56052.000000


In [8]:
fac_df.groupby('Is Public').describe()

Assistant Professors                                                \
                         count          mean          std      min       25%   
Is Public                                                                      
False                     13.0  56830.846154  8462.767857  47133.0  51606.00   
True                      12.0  60209.250000  7144.154688  52839.0  56229.75   

                                     Associate Professors                \
               50%      75%      max                count          mean   
Is Public                                                                 
False      53388.0  57816.0  72702.0                 13.0  66135.461538   
True       57528.0  61047.0  77157.0                 12.0  64632.750000   

            ...    Professors           Unranked                        \
            ...           75%       max    count     mean          std   
Is Public   ...                                                          
False       ...      83034.00  118953.0      2.0  51552.0  6363.961031   
True        ...      78662.25  113094.0      1.0  34407.0          NaN   

                                                        
               min      25%      50%      75%      max  
Is Public                                               
False      47052.0  49302.0  51552.0  53802.0  56052.0  
True       34407.0  34407.0  34407.0  34407.0  34407.0  

[2 rows x 48 columns]

# Minnesota colleges

The file minnesota_colleges.txt contains data on 10 universities and colleges in Minnesota.  (Source:  Wikipedia.)  

In [9]:
# Change the value of fileLoc to match where you saved the file.
newfileLoc = "minnesota_colleges.txt"
with open(newfileLoc) as mn_file:
    line_num = 0
    for line in mn_file:
        print("Line number:", line_num, line)
        line_num +=1
        if line_num==25:
            break

Line number: 0 Institution:  University of Minnesota Twin Cities

Line number: 1 Motto	Commune vinculum omnibus artibus (Latin)

Line number: 2 Motto in English

Line number: 3 A common bond for all the arts

Line number: 4 Type	Public

Line number: 5 Flagship University

Line number: 6 Land-Grant

Line number: 7 Sea-Grant

Line number: 8 Space-Grant

Line number: 9 Established	1851

Line number: 10 

Line number: 11 Institution:  Carleton College

Line number: 12 Carletoncollegeseal.png

Line number: 13 Latin: Collegium Carleton

Line number: 14 Former names

Line number: 15 Northfield College

Line number: 16 Motto	Declaratio Sermonum Tuorum Illuminat (Latin)

Line number: 17 Motto in English

Line number: 18 The Revelation / Announcement of Your Words Illuminates

Line number: 19 Type	Private liberal arts college, Established	1866

Line number: 20 

Line number: 21 Institution:  William Mitchell College of Law

Line number: 22 WilliamMitchell.png

Line number: 23 Motto	Practical Wis

We'd like to make a data frame of each institution's name and the year when it was established, but there's not a consistent pattern of how many lines of text represent each institution.  (For example, we can't assume that every 10th line contains the name of the institution.)  This is a common challenge when working with large data sets.

One option would be to use Python to identify which lines have unexpected information, and manually edit those lines in a text editor, such as Notepad++.

Another option is to use the fact that each institution's name is preceded by "Institution:" to identify where each institution's data starts:

In [10]:
with open(newfileLoc) as mn_file:
    line_num = 0
    name = ""
    dict_list = []
    column_names = ["Name", "Info"]
    
    while line_num < 25:
        nextLine = mn_file.readline()
        line_num +=1
        
        # Check whether nextLine contains "Institution:  "
        instStart = nextLine.find('Institution:')
        # any value other than -1 indicates that we found the string
        if instStart != -1: 
            if name:
                # Store the previous institution's data in the list of dictionaries
                dict_list.append(dict(zip(column_names, [name, info])))
            name = nextLine.strip("Institution:  ")
            name = name.strip()
            # This is a new institution, so clear out the info variable
            info = ""
        else: # nextLine does not contain "Institution:  "
            info = info + nextLine

coll_df = pd.DataFrame(dict_list)
coll_df

,Info,Name
0,Motto\tCommune vinculum omnibus artibus (Latin...,University of Minnesota Twin Cities
1,Carletoncollegeseal.png\nLatin: Collegium Carl...,Carleton College


Next, we need to modify our code to look for the word "Established".

In [11]:
with open(newfileLoc) as mn_file:
    line_num = 0
    name = ""
    dict_list = []
    column_names = ["Name", "Established"]
    
    while line_num < 25:
        nextLine = mn_file.readline()
        line_num +=1
        
        # Check whether nextLine contains "Institution:  " or "Established"
        instStart = nextLine.find('Institution:')
        estStart = nextLine.find('Established')
        # any value other than -1 indicates that we found the string
        if instStart != -1: 
            if name:
                # Store the previous institution's data in the list of dictionaries
                dict_list.append(dict(zip(column_names, [name, establishedYear])))
            name = nextLine.strip("Institution:  ")
            name = name.strip()
        elif estStart != -1:
            # Read the year established, ignoring anything on the line that comes before "Established"
            establishedYear = nextLine[estStart:]
            # Strip out extraneous parts of the line
            establishedYear = establishedYear.strip("Established")
            establishedYear = establishedYear.strip()
            # Convert to numeric data
            establishedYear = eval(establishedYear)

coll_df = pd.DataFrame(dict_list)
coll_df

,Established,Name
0,1851,University of Minnesota Twin Cities
1,1866,Carleton College


Finally, insert a break if we hit the end of the file, and run the code on the entire file:

In [12]:
with open(newfileLoc) as mn_file:
    line_num = 0
    name = ""
    dict_list = []
    column_names = ["Name", "Established"]
    
    while True:
        nextLine = mn_file.readline()
        if not nextLine:
            # Store the last institution's data in the list of dictionaries
            dict_list.append(dict(zip(column_names, [name, establishedYear])))
            break
        line_num +=1
        
        # Check whether nextLine contains "Institution:  " or "Established"
        instStart = nextLine.find('Institution:')
        estStart = nextLine.find('Established')
        # any value other than -1 indicates that we found the string
        if instStart != -1: 
            if name:
                # Store the previous institution's data in the list of dictionaries
                dict_list.append(dict(zip(column_names, [name, establishedYear])))
            name = nextLine.strip("Institution:  ")
            name = name.strip()
        elif estStart != -1:
            # Read the year established, ignoring anything on the line that comes before "Established"
            establishedYear = nextLine[estStart:]
            # Strip out extraneous parts of the line
            establishedYear = establishedYear.strip("Established")
            establishedYear = establishedYear.strip()
            # Convert to numeric data
            establishedYear = eval(establishedYear)

coll_df = pd.DataFrame(dict_list)
coll_df

,Established,Name
0,1851,University of Minnesota Twin Cities
1,1866,Carleton College
2,1956,William Mitchell College of Law
3,1874,Macalester College
4,1885,University of St. Thomas
5,1874,St. Olaf College
6,1902,University of Minnesota Duluth
7,1868,"Minnesota State University, Mankato"
8,1971,Metropolitan State University
9,1913,College of Saint Benedict
